In [1]:
from pathlib import Path
import os

PROJ_ROOT = Path().resolve().parent
KEYS_DIR = PROJ_ROOT / 'keys' 
keys = KEYS_DIR / 'socos-quickstep-601-403a611fa9db.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(keys)

In [2]:
    # PROCESS
    # 1. Extract tables from Big Query using a JOIN SQL query - combining the Binary Answers on the 'names' with the Questions
    #  - it is important to NOT use the Learner information from the JiveLearner to create a 'Learner' dataframe, because it results in fewer data points. Instead use a multiple-join
    # 2. Read in the data as a dateframe
    # 4. Process the data by:
    #     - choosing the last modified
    #     - eliminate duplicates
    # 5. create a new dataframe which has the Learners as the index and Questions as columns
    #     - encode the categorical answers numerically (-2, -1, 0, 1, 2)
    #     - changing the answers to overwrite based on the most definitive (highest or lowest) number
    # 4. export the table to bigquery
    #     - None / NaN BQ datatype

    # TOP QUESTIONS FOR TROUBLESHOOTING
    # Q 371: "Do you work outside of the home?" | 2,012
    # Q 367: "Does {kid_name} attend school outside of the home?" | 1,349
    # Q 368: "Do you have reliable childcare for {kid_name}?" | 1,216
    # Q 24: "Do you know the name of {kid_name}'s primary doctor?" | 350
    # Q 13: "Do you know your next door neighbor's first name?" | 341
    # Q 25: "Does your family have health care?" | 339
    # Q 18: "Does your spouse work multiple jobs?" | 335
    # Q 590: "Do you have a primary doctor?" | 325
    # Q 15: "Does your spouse work full-time?" | 318
    # Q 41: "Are you the first person in your family to graduate college?" | 314

    # LIST OF POSSIBLE ANSWERS
    # {'Dislike',
    #  "Doesn't apply",
    #  'Kid is too old',
    #  'Kid is too young',
    #  'No',
    #  'Not sure',
    #  'Yes',
    #  'absolutely',
    #  'abstract',
    #  'barely',
    #  'definitely',
    #  "doesn't apply",
    #  'good idea',
    #  'interesting',
    #  'kid answered',
    #  'kid is too old',
    #  'kid is too young',
    #  'kid too old',
    #  'kid too young',
    #  'no',
    #  'nope',
    #  'not sure',
    #  'not yet',
    #  'of course',
    #  'personal',
    #  'philosophical',
    #  'probably',
    #  'somewhat',
    #  'yes'}


import pandas as pd
from google.cloud import bigquery


In [3]:
#instantiate client
project = 'socos-quickstep-601'
client = bigquery.Client(project=project)

join_query="""
#standardSQL
SELECT a.JiveLearner.id as learner, \
b.message.jive_id as question, \
a.answer, a.last_modified, \
a.JiveEnvelope.name, \
b.message.learner_name, \
a.tags.tag, \
a.tags.metatag, \
c.who, \
c.text, \
c.answer_type, ARRAY_TO_STRING(a.tags.tag, ', ') as answer_tag, ARRAY_TO_STRING(a.tags.metatag, ', ') as metatags
FROM `socos-quickstep-601.museathome.BinaryQuestionAnswer` as a
JOIN `socos-quickstep-601.museathome.JiveEnvelope` as b ON a.JiveEnvelope.name=b.__key__.name
JOIN `socos-quickstep-601.museathome.questions` as c ON CAST(b.message.jive_id AS NUMERIC) = c.id
ORDER BY a.JiveLearner.id, b.message.learner_name, question ASC
"""

job_config = bigquery.QueryJobConfig()
join_query_job = client.query(join_query, location = 'US')
res = join_query_job.result()

#create dataframes
combined_df = res.to_dataframe()
combined_true = combined_df[combined_df['answer'] == True]
combined_false = combined_df[combined_df['answer'] == False]

#only select the last modified from all of the keys with the same name
combined_df = combined_df.sort_values('last_modified').drop_duplicates(subset='name',keep='last')

#rows of final df
learner_df = combined_df[['learner']]
learner_df.reset_index(inplace=True, drop=True)
learner_df = learner_df.drop_duplicates(keep='last')
learner_df = learner_df.set_index('learner')

#there are fewer questions answered
#than there are questions in the question database
#so we are going to query the questions directly from the questions database

questions_query="""
#standardSQL
SELECT * FROM `socos-quickstep-601.museathome.questions`
"""
job_config = bigquery.QueryJobConfig()
questions_query_job = client.query(questions_query, location = 'US')
res = questions_query_job.result()
questions_df = res.to_dataframe()

questions_df = questions_df[['id']].drop_duplicates(keep='last')
questions_df = questions_df.reset_index(drop=True)

#create muse_model_df
dummy = pd.get_dummies(questions_df['id'])
muse_model_df = pd.concat([learner_df, dummy])

#only selects all the Musers
muse_model_df = muse_model_df[:2967]


TransportError: HTTPSConnectionPool(host='oauth2.googleapis.com', port=443): Max retries exceeded with url: /token (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11a537f98>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [4]:
print('The length of the join query is: ' + str(combined_df.shape[0]))

NameError: name 'combined_df' is not defined

In [21]:
def process_true(df, muse_model_df):
    #True answers
    for i in df.index:
        learner = df.at[i, 'learner']
        question = df.at[i, 'question']
        question = int(question)
        muse_model_df.loc[learner, question] = 1

def process_false(df, muse_model_df):        
    #False answers
    for i in df.index:
        learner = df.at[i, 'learner']
        question = df.at[i, 'question']
        question = int(question)
        muse_model_df.loc[learner, question] = -1

In [6]:
#create new 'final_answer' column of only the answers
for i in combined_df.index:
    if 'answer' in combined_df.at[i, 'metatag']:
        combined_df.at[i, 'final_answer'] = combined_df.at[i, 'tag'][0]

In [20]:
combined_df['final_answer'].value_counts().sum()

15563

In [8]:
# Get only answers that don't have True/False answers and are not null in the tags
combined_tags_final = combined_df[combined_df.final_answer.notnull() & combined_df.answer.isnull()]

In [35]:
# Dictionary for numbers
def switcher(ans):
    answers = {
        'yes': 1,
        'no': -1,
        'definitely': 2,
        'probably': 1,
        'somewhat': 0,
        'barely': -1,
        'nope': -2
    }
    
    return answers.get(ans, None)

def add_tags(df, muse_model_df):
    for i in df.index:    
        answer = df.at[i, 'final_answer'].lower()
        learner = df.at[i, 'learner']
        question = df.at[i, 'question']
        question = int(question) 
        
        muse_answer = switcher(answer)
        muse_model_df.loc[learner, question] = muse_answer
        
        return

In [22]:
process_true(combined_true, muse_model_df)
process_false(combined_false, muse_model_df)

In [36]:
add_tags(combined_tags_final, muse_model_df)

In [37]:
muse_model_df.count().sum()

21610

In [29]:
muse_model_df[367].value_counts()

 1.0    27
-1.0     2
Name: 367, dtype: int64

In [13]:
#total number of answers without answer tags is 468
#total number of answers not in muse_model from combined_df is 1067

In [14]:
34355 - 33288

1067

In [15]:
1067 - 468

599

In [16]:
# Must add letter in front of columns to upload to Big Query
new_cols = list(muse_model_df.columns)

for i in range(len(new_cols)):
    word = str(new_cols[i])
    new_cols[i] = 'q' + word

In [19]:
muse_model_df.count().sum()

21610

In [18]:
project = 'socos-quickstep-601'
dataset_id = 'muse_matrix'
table_id = 'muse_matrix.learners_questions'

# Load job_config and define schema
job_config = bigquery.LoadJobConfig(
        schema=[
            bigquery.SchemaField("q1", "INTEGER"),
            bigquery.SchemaField("q2", "INTEGER"),
            bigquery.SchemaField("q3", "INTEGER"),
            bigquery.SchemaField("q4", "INTEGER"),
            bigquery.SchemaField("q5", "INTEGER"),
            bigquery.SchemaField("q6", "INTEGER"),
            bigquery.SchemaField("q7", "INTEGER"),
            bigquery.SchemaField("q8", "INTEGER"),
            bigquery.SchemaField("q9", "INTEGER"),
            bigquery.SchemaField("q10", "INTEGER"),
            bigquery.SchemaField("q11", "INTEGER"),
            bigquery.SchemaField("q12", "INTEGER"),
            bigquery.SchemaField("q13", "INTEGER"),
            bigquery.SchemaField("q14", "INTEGER"),
            bigquery.SchemaField("q15", "INTEGER"),
            bigquery.SchemaField("q16", "INTEGER"),
            bigquery.SchemaField("q17", "INTEGER"),
            bigquery.SchemaField("q18", "INTEGER"),
            bigquery.SchemaField("q19", "INTEGER"),
            bigquery.SchemaField("q20", "INTEGER"),
            bigquery.SchemaField("q21", "INTEGER"),
            bigquery.SchemaField("q22", "INTEGER"),
            bigquery.SchemaField("q23", "INTEGER"),
            bigquery.SchemaField("q24", "INTEGER"),
            bigquery.SchemaField("q25", "INTEGER"),
            bigquery.SchemaField("q26", "INTEGER"),
            bigquery.SchemaField("q27", "INTEGER"),
            bigquery.SchemaField("q28", "INTEGER"),
            bigquery.SchemaField("q29", "INTEGER"),
            bigquery.SchemaField("q30", "INTEGER"),
            bigquery.SchemaField("q31", "INTEGER"),
            bigquery.SchemaField("q32", "INTEGER"),
            bigquery.SchemaField("q33", "INTEGER"),
            bigquery.SchemaField("q34", "INTEGER"),
            bigquery.SchemaField("q35", "INTEGER"),
            bigquery.SchemaField("q36", "INTEGER"),
            bigquery.SchemaField("q37", "INTEGER"),
            bigquery.SchemaField("q38", "INTEGER"),
            bigquery.SchemaField("q39", "INTEGER"),
            bigquery.SchemaField("q40", "INTEGER"),
            bigquery.SchemaField("q41", "INTEGER"),
            bigquery.SchemaField("q42", "INTEGER"),
            bigquery.SchemaField("q43", "INTEGER"),
            bigquery.SchemaField("q44", "INTEGER"),
            bigquery.SchemaField("q45", "INTEGER"),
            bigquery.SchemaField("q46", "INTEGER"),
            bigquery.SchemaField("q47", "INTEGER"),
            bigquery.SchemaField("q48", "INTEGER"),
            bigquery.SchemaField("q49", "INTEGER"),
            bigquery.SchemaField("q50", "INTEGER"),
            bigquery.SchemaField("q51", "INTEGER"),
            bigquery.SchemaField("q52", "INTEGER"),
            bigquery.SchemaField("q53", "INTEGER"),
            bigquery.SchemaField("q54", "INTEGER"),
            bigquery.SchemaField("q55", "INTEGER"),
            bigquery.SchemaField("q56", "INTEGER"),
            bigquery.SchemaField("q57", "INTEGER"),
            bigquery.SchemaField("q58", "INTEGER"),
            bigquery.SchemaField("q59", "INTEGER"),
            bigquery.SchemaField("q60", "INTEGER"),
            bigquery.SchemaField("q61", "INTEGER"),
            bigquery.SchemaField("q62", "INTEGER"),
            bigquery.SchemaField("q63", "INTEGER"),
            bigquery.SchemaField("q64", "INTEGER"),
            bigquery.SchemaField("q65", "INTEGER"),
            bigquery.SchemaField("q66", "INTEGER"),
            bigquery.SchemaField("q67", "INTEGER"),
            bigquery.SchemaField("q68", "INTEGER"),
            bigquery.SchemaField("q69", "INTEGER"),
            bigquery.SchemaField("q70", "INTEGER"),
            bigquery.SchemaField("q71", "INTEGER"),
            bigquery.SchemaField("q72", "INTEGER"),
            bigquery.SchemaField("q73", "INTEGER"),
            bigquery.SchemaField("q74", "INTEGER"),
            bigquery.SchemaField("q75", "INTEGER"),
            bigquery.SchemaField("q76", "INTEGER"),
            bigquery.SchemaField("q77", "INTEGER"),
            bigquery.SchemaField("q78", "INTEGER"),
            bigquery.SchemaField("q79", "INTEGER"),
            bigquery.SchemaField("q80", "INTEGER"),
            bigquery.SchemaField("q81", "INTEGER"),
            bigquery.SchemaField("q82", "INTEGER"),
            bigquery.SchemaField("q83", "INTEGER"),
            bigquery.SchemaField("q84", "INTEGER"),
            bigquery.SchemaField("q85", "INTEGER"),
            bigquery.SchemaField("q86", "INTEGER"),
            bigquery.SchemaField("q87", "INTEGER"),
            bigquery.SchemaField("q88", "INTEGER"),
            bigquery.SchemaField("q89", "INTEGER"),
            bigquery.SchemaField("q90", "INTEGER"),
            bigquery.SchemaField("q91", "INTEGER"),
            bigquery.SchemaField("q92", "INTEGER"),
            bigquery.SchemaField("q93", "INTEGER"),
            bigquery.SchemaField("q94", "INTEGER"),
            bigquery.SchemaField("q95", "INTEGER"),
            bigquery.SchemaField("q96", "INTEGER"),
            bigquery.SchemaField("q97", "INTEGER"),
            bigquery.SchemaField("q98", "INTEGER"),
            bigquery.SchemaField("q99", "INTEGER"),
            bigquery.SchemaField("q100", "INTEGER"),
            bigquery.SchemaField("q101", "INTEGER"),
            bigquery.SchemaField("q102", "INTEGER"),
            bigquery.SchemaField("q103", "INTEGER"),
            bigquery.SchemaField("q104", "INTEGER"),
            bigquery.SchemaField("q105", "INTEGER"),
            bigquery.SchemaField("q106", "INTEGER"),
            bigquery.SchemaField("q107", "INTEGER"),
            bigquery.SchemaField("q108", "INTEGER"),
            bigquery.SchemaField("q109", "INTEGER"),
            bigquery.SchemaField("q110", "INTEGER"),
            bigquery.SchemaField("q111", "INTEGER"),
            bigquery.SchemaField("q112", "INTEGER"),
            bigquery.SchemaField("q113", "INTEGER"),
            bigquery.SchemaField("q114", "INTEGER"),
            bigquery.SchemaField("q115", "INTEGER"),
            bigquery.SchemaField("q116", "INTEGER"),
            bigquery.SchemaField("q117", "INTEGER"),
            bigquery.SchemaField("q118", "INTEGER"),
            bigquery.SchemaField("q119", "INTEGER"),
            bigquery.SchemaField("q120", "INTEGER"),
            bigquery.SchemaField("q121", "INTEGER"),
            bigquery.SchemaField("q122", "INTEGER"),
            bigquery.SchemaField("q123", "INTEGER"),
            bigquery.SchemaField("q124", "INTEGER"),
            bigquery.SchemaField("q125", "INTEGER"),
            bigquery.SchemaField("q126", "INTEGER"),
            bigquery.SchemaField("q127", "INTEGER"),
            bigquery.SchemaField("q128", "INTEGER"),
            bigquery.SchemaField("q129", "INTEGER"),
            bigquery.SchemaField("q130", "INTEGER"),
            bigquery.SchemaField("q131", "INTEGER"),
            bigquery.SchemaField("q132", "INTEGER"),
            bigquery.SchemaField("q133", "INTEGER"),
            bigquery.SchemaField("q134", "INTEGER"),
            bigquery.SchemaField("q135", "INTEGER"),
            bigquery.SchemaField("q136", "INTEGER"),
            bigquery.SchemaField("q137", "INTEGER"),
            bigquery.SchemaField("q138", "INTEGER"),
            bigquery.SchemaField("q139", "INTEGER"),
            bigquery.SchemaField("q140", "INTEGER"),
            bigquery.SchemaField("q141", "INTEGER"),
            bigquery.SchemaField("q142", "INTEGER"),
            bigquery.SchemaField("q143", "INTEGER"),
            bigquery.SchemaField("q144", "INTEGER"),
            bigquery.SchemaField("q145", "INTEGER"),
            bigquery.SchemaField("q146", "INTEGER"),
            bigquery.SchemaField("q147", "INTEGER"),
            bigquery.SchemaField("q148", "INTEGER"),
            bigquery.SchemaField("q149", "INTEGER"),
            bigquery.SchemaField("q150", "INTEGER"),
            bigquery.SchemaField("q151", "INTEGER"),
            bigquery.SchemaField("q152", "INTEGER"),
            bigquery.SchemaField("q153", "INTEGER"),
            bigquery.SchemaField("q154", "INTEGER"),
            bigquery.SchemaField("q155", "INTEGER"),
            bigquery.SchemaField("q156", "INTEGER"),
            bigquery.SchemaField("q157", "INTEGER"),
            bigquery.SchemaField("q158", "INTEGER"),
            bigquery.SchemaField("q159", "INTEGER"),
            bigquery.SchemaField("q160", "INTEGER"),
            bigquery.SchemaField("q161", "INTEGER"),
            bigquery.SchemaField("q162", "INTEGER"),
            bigquery.SchemaField("q163", "INTEGER"),
            bigquery.SchemaField("q164", "INTEGER"),
            bigquery.SchemaField("q165", "INTEGER"),
            bigquery.SchemaField("q166", "INTEGER"),
            bigquery.SchemaField("q167", "INTEGER"),
            bigquery.SchemaField("q168", "INTEGER"),
            bigquery.SchemaField("q169", "INTEGER"),
            bigquery.SchemaField("q170", "INTEGER"),
            bigquery.SchemaField("q171", "INTEGER"),
            bigquery.SchemaField("q172", "INTEGER"),
            bigquery.SchemaField("q173", "INTEGER"),
            bigquery.SchemaField("q174", "INTEGER"),
            bigquery.SchemaField("q175", "INTEGER"),
            bigquery.SchemaField("q176", "INTEGER"),
            bigquery.SchemaField("q177", "INTEGER"),
            bigquery.SchemaField("q178", "INTEGER"),
            bigquery.SchemaField("q179", "INTEGER"),
            bigquery.SchemaField("q180", "INTEGER"),
            bigquery.SchemaField("q181", "INTEGER"),
            bigquery.SchemaField("q182", "INTEGER"),
            bigquery.SchemaField("q183", "INTEGER"),
            bigquery.SchemaField("q184", "INTEGER"),
            bigquery.SchemaField("q185", "INTEGER"),
            bigquery.SchemaField("q186", "INTEGER"),
            bigquery.SchemaField("q187", "INTEGER"),
            bigquery.SchemaField("q188", "INTEGER"),
            bigquery.SchemaField("q189", "INTEGER"),
            bigquery.SchemaField("q190", "INTEGER"),
            bigquery.SchemaField("q191", "INTEGER"),
            bigquery.SchemaField("q192", "INTEGER"),
            bigquery.SchemaField("q193", "INTEGER"),
            bigquery.SchemaField("q194", "INTEGER"),
            bigquery.SchemaField("q195", "INTEGER"),
            bigquery.SchemaField("q196", "INTEGER"),
            bigquery.SchemaField("q197", "INTEGER"),
            bigquery.SchemaField("q198", "INTEGER"),
            bigquery.SchemaField("q199", "INTEGER"),
            bigquery.SchemaField("q200", "INTEGER"),
            bigquery.SchemaField("q201", "INTEGER"),
            bigquery.SchemaField("q202", "INTEGER"),
            bigquery.SchemaField("q203", "INTEGER"),
            bigquery.SchemaField("q204", "INTEGER"),
            bigquery.SchemaField("q205", "INTEGER"),
            bigquery.SchemaField("q206", "INTEGER"),
            bigquery.SchemaField("q207", "INTEGER"),
            bigquery.SchemaField("q208", "INTEGER"),
            bigquery.SchemaField("q209", "INTEGER"),
            bigquery.SchemaField("q210", "INTEGER"),
            bigquery.SchemaField("q211", "INTEGER"),
            bigquery.SchemaField("q212", "INTEGER"),
            bigquery.SchemaField("q213", "INTEGER"),
            bigquery.SchemaField("q214", "INTEGER"),
            bigquery.SchemaField("q215", "INTEGER"),
            bigquery.SchemaField("q216", "INTEGER"),
            bigquery.SchemaField("q217", "INTEGER"),
            bigquery.SchemaField("q218", "INTEGER"),
            bigquery.SchemaField("q219", "INTEGER"),
            bigquery.SchemaField("q220", "INTEGER"),
            bigquery.SchemaField("q221", "INTEGER"),
            bigquery.SchemaField("q222", "INTEGER"),
            bigquery.SchemaField("q223", "INTEGER"),
            bigquery.SchemaField("q224", "INTEGER"),
            bigquery.SchemaField("q225", "INTEGER"),
            bigquery.SchemaField("q226", "INTEGER"),
            bigquery.SchemaField("q227", "INTEGER"),
            bigquery.SchemaField("q228", "INTEGER"),
            bigquery.SchemaField("q229", "INTEGER"),
            bigquery.SchemaField("q230", "INTEGER"),
            bigquery.SchemaField("q231", "INTEGER"),
            bigquery.SchemaField("q232", "INTEGER"),
            bigquery.SchemaField("q233", "INTEGER"),
            bigquery.SchemaField("q234", "INTEGER"),
            bigquery.SchemaField("q235", "INTEGER"),
            bigquery.SchemaField("q236", "INTEGER"),
            bigquery.SchemaField("q237", "INTEGER"),
            bigquery.SchemaField("q238", "INTEGER"),
            bigquery.SchemaField("q239", "INTEGER"),
            bigquery.SchemaField("q240", "INTEGER"),
            bigquery.SchemaField("q241", "INTEGER"),
            bigquery.SchemaField("q242", "INTEGER"),
            bigquery.SchemaField("q243", "INTEGER"),
            bigquery.SchemaField("q244", "INTEGER"),
            bigquery.SchemaField("q245", "INTEGER"),
            bigquery.SchemaField("q246", "INTEGER"),
            bigquery.SchemaField("q247", "INTEGER"),
            bigquery.SchemaField("q248", "INTEGER"),
            bigquery.SchemaField("q249", "INTEGER"),
            bigquery.SchemaField("q250", "INTEGER"),
            bigquery.SchemaField("q251", "INTEGER"),
            bigquery.SchemaField("q252", "INTEGER"),
            bigquery.SchemaField("q253", "INTEGER"),
            bigquery.SchemaField("q254", "INTEGER"),
            bigquery.SchemaField("q255", "INTEGER"),
            bigquery.SchemaField("q256", "INTEGER"),
            bigquery.SchemaField("q257", "INTEGER"),
            bigquery.SchemaField("q258", "INTEGER"),
            bigquery.SchemaField("q259", "INTEGER"),
            bigquery.SchemaField("q260", "INTEGER"),
            bigquery.SchemaField("q261", "INTEGER"),
            bigquery.SchemaField("q262", "INTEGER"),
            bigquery.SchemaField("q263", "INTEGER"),
            bigquery.SchemaField("q264", "INTEGER"),
            bigquery.SchemaField("q265", "INTEGER"),
            bigquery.SchemaField("q266", "INTEGER"),
            bigquery.SchemaField("q267", "INTEGER"),
            bigquery.SchemaField("q268", "INTEGER"),
            bigquery.SchemaField("q269", "INTEGER"),
            bigquery.SchemaField("q270", "INTEGER"),
            bigquery.SchemaField("q271", "INTEGER"),
            bigquery.SchemaField("q272", "INTEGER"),
            bigquery.SchemaField("q273", "INTEGER"),
            bigquery.SchemaField("q274", "INTEGER"),
            bigquery.SchemaField("q275", "INTEGER"),
            bigquery.SchemaField("q276", "INTEGER"),
            bigquery.SchemaField("q277", "INTEGER"),
            bigquery.SchemaField("q278", "INTEGER"),
            bigquery.SchemaField("q279", "INTEGER"),
            bigquery.SchemaField("q280", "INTEGER"),
            bigquery.SchemaField("q281", "INTEGER"),
            bigquery.SchemaField("q282", "INTEGER"),
            bigquery.SchemaField("q283", "INTEGER"),
            bigquery.SchemaField("q284", "INTEGER"),
            bigquery.SchemaField("q285", "INTEGER"),
            bigquery.SchemaField("q286", "INTEGER"),
            bigquery.SchemaField("q287", "INTEGER"),
            bigquery.SchemaField("q288", "INTEGER"),
            bigquery.SchemaField("q289", "INTEGER"),
            bigquery.SchemaField("q290", "INTEGER"),
            bigquery.SchemaField("q291", "INTEGER"),
            bigquery.SchemaField("q292", "INTEGER"),
            bigquery.SchemaField("q293", "INTEGER"),
            bigquery.SchemaField("q294", "INTEGER"),
            bigquery.SchemaField("q295", "INTEGER"),
            bigquery.SchemaField("q296", "INTEGER"),
            bigquery.SchemaField("q297", "INTEGER"),
            bigquery.SchemaField("q298", "INTEGER"),
            bigquery.SchemaField("q299", "INTEGER"),
            bigquery.SchemaField("q300", "INTEGER"),
            bigquery.SchemaField("q301", "INTEGER"),
            bigquery.SchemaField("q302", "INTEGER"),
            bigquery.SchemaField("q303", "INTEGER"),
            bigquery.SchemaField("q304", "INTEGER"),
            bigquery.SchemaField("q305", "INTEGER"),
            bigquery.SchemaField("q306", "INTEGER"),
            bigquery.SchemaField("q307", "INTEGER"),
            bigquery.SchemaField("q308", "INTEGER"),
            bigquery.SchemaField("q309", "INTEGER"),
            bigquery.SchemaField("q310", "INTEGER"),
            bigquery.SchemaField("q311", "INTEGER"),
            bigquery.SchemaField("q312", "INTEGER"),
            bigquery.SchemaField("q313", "INTEGER"),
            bigquery.SchemaField("q314", "INTEGER"),
            bigquery.SchemaField("q315", "INTEGER"),
            bigquery.SchemaField("q316", "INTEGER"),
            bigquery.SchemaField("q317", "INTEGER"),
            bigquery.SchemaField("q318", "INTEGER"),
            bigquery.SchemaField("q319", "INTEGER"),
            bigquery.SchemaField("q320", "INTEGER"),
            bigquery.SchemaField("q321", "INTEGER"),
            bigquery.SchemaField("q322", "INTEGER"),
            bigquery.SchemaField("q323", "INTEGER"),
            bigquery.SchemaField("q324", "INTEGER"),
            bigquery.SchemaField("q325", "INTEGER"),
            bigquery.SchemaField("q326", "INTEGER"),
            bigquery.SchemaField("q327", "INTEGER"),
            bigquery.SchemaField("q328", "INTEGER"),
            bigquery.SchemaField("q329", "INTEGER"),
            bigquery.SchemaField("q330", "INTEGER"),
            bigquery.SchemaField("q331", "INTEGER"),
            bigquery.SchemaField("q332", "INTEGER"),
            bigquery.SchemaField("q333", "INTEGER"),
            bigquery.SchemaField("q334", "INTEGER"),
            bigquery.SchemaField("q335", "INTEGER"),
            bigquery.SchemaField("q336", "INTEGER"),
            bigquery.SchemaField("q337", "INTEGER"),
            bigquery.SchemaField("q338", "INTEGER"),
            bigquery.SchemaField("q339", "INTEGER"),
            bigquery.SchemaField("q340", "INTEGER"),
            bigquery.SchemaField("q341", "INTEGER"),
            bigquery.SchemaField("q342", "INTEGER"),
            bigquery.SchemaField("q343", "INTEGER"),
            bigquery.SchemaField("q344", "INTEGER"),
            bigquery.SchemaField("q345", "INTEGER"),
            bigquery.SchemaField("q346", "INTEGER"),
            bigquery.SchemaField("q347", "INTEGER"),
            bigquery.SchemaField("q348", "INTEGER"),
            bigquery.SchemaField("q349", "INTEGER"),
            bigquery.SchemaField("q350", "INTEGER"),
            bigquery.SchemaField("q351", "INTEGER"),
            bigquery.SchemaField("q352", "INTEGER"),
            bigquery.SchemaField("q353", "INTEGER"),
            bigquery.SchemaField("q354", "INTEGER"),
            bigquery.SchemaField("q355", "INTEGER"),
            bigquery.SchemaField("q356", "INTEGER"),
            bigquery.SchemaField("q357", "INTEGER"),
            bigquery.SchemaField("q358", "INTEGER"),
            bigquery.SchemaField("q359", "INTEGER"),
            bigquery.SchemaField("q360", "INTEGER"),
            bigquery.SchemaField("q361", "INTEGER"),
            bigquery.SchemaField("q362", "INTEGER"),
            bigquery.SchemaField("q363", "INTEGER"),
            bigquery.SchemaField("q364", "INTEGER"),
            bigquery.SchemaField("q365", "INTEGER"),
            bigquery.SchemaField("q366", "INTEGER"),
            bigquery.SchemaField("q367", "INTEGER"),
            bigquery.SchemaField("q368", "INTEGER"),
            bigquery.SchemaField("q369", "INTEGER"),
            bigquery.SchemaField("q370", "INTEGER"),
            bigquery.SchemaField("q371", "INTEGER"),
            bigquery.SchemaField("q372", "INTEGER"),
            bigquery.SchemaField("q373", "INTEGER"),
            bigquery.SchemaField("q374", "INTEGER"),
            bigquery.SchemaField("q375", "INTEGER"),
            bigquery.SchemaField("q376", "INTEGER"),
            bigquery.SchemaField("q377", "INTEGER"),
            bigquery.SchemaField("q378", "INTEGER"),
            bigquery.SchemaField("q379", "INTEGER"),
            bigquery.SchemaField("q380", "INTEGER"),
            bigquery.SchemaField("q381", "INTEGER"),
            bigquery.SchemaField("q382", "INTEGER"),
            bigquery.SchemaField("q383", "INTEGER"),
            bigquery.SchemaField("q384", "INTEGER"),
            bigquery.SchemaField("q385", "INTEGER"),
            bigquery.SchemaField("q386", "INTEGER"),
            bigquery.SchemaField("q387", "INTEGER"),
            bigquery.SchemaField("q388", "INTEGER"),
            bigquery.SchemaField("q389", "INTEGER"),
            bigquery.SchemaField("q390", "INTEGER"),
            bigquery.SchemaField("q391", "INTEGER"),
            bigquery.SchemaField("q392", "INTEGER"),
            bigquery.SchemaField("q393", "INTEGER"),
            bigquery.SchemaField("q394", "INTEGER"),
            bigquery.SchemaField("q395", "INTEGER"),
            bigquery.SchemaField("q396", "INTEGER"),
            bigquery.SchemaField("q397", "INTEGER"),
            bigquery.SchemaField("q398", "INTEGER"),
            bigquery.SchemaField("q399", "INTEGER"),
            bigquery.SchemaField("q400", "INTEGER"),
            bigquery.SchemaField("q401", "INTEGER"),
            bigquery.SchemaField("q402", "INTEGER"),
            bigquery.SchemaField("q403", "INTEGER"),
            bigquery.SchemaField("q404", "INTEGER"),
            bigquery.SchemaField("q405", "INTEGER"),
            bigquery.SchemaField("q406", "INTEGER"),
            bigquery.SchemaField("q407", "INTEGER"),
            bigquery.SchemaField("q408", "INTEGER"),
            bigquery.SchemaField("q409", "INTEGER"),
            bigquery.SchemaField("q410", "INTEGER"),
            bigquery.SchemaField("q411", "INTEGER"),
            bigquery.SchemaField("q412", "INTEGER"),
            bigquery.SchemaField("q413", "INTEGER"),
            bigquery.SchemaField("q414", "INTEGER"),
            bigquery.SchemaField("q415", "INTEGER"),
            bigquery.SchemaField("q416", "INTEGER"),
            bigquery.SchemaField("q417", "INTEGER"),
            bigquery.SchemaField("q418", "INTEGER"),
            bigquery.SchemaField("q419", "INTEGER"),
            bigquery.SchemaField("q420", "INTEGER"),
            bigquery.SchemaField("q421", "INTEGER"),
            bigquery.SchemaField("q422", "INTEGER"),
            bigquery.SchemaField("q423", "INTEGER"),
            bigquery.SchemaField("q424", "INTEGER"),
            bigquery.SchemaField("q425", "INTEGER"),
            bigquery.SchemaField("q426", "INTEGER"),
            bigquery.SchemaField("q427", "INTEGER"),
            bigquery.SchemaField("q428", "INTEGER"),
            bigquery.SchemaField("q429", "INTEGER"),
            bigquery.SchemaField("q430", "INTEGER"),
            bigquery.SchemaField("q431", "INTEGER"),
            bigquery.SchemaField("q432", "INTEGER"),
            bigquery.SchemaField("q433", "INTEGER"),
            bigquery.SchemaField("q434", "INTEGER"),
            bigquery.SchemaField("q435", "INTEGER"),
            bigquery.SchemaField("q436", "INTEGER"),
            bigquery.SchemaField("q437", "INTEGER"),
            bigquery.SchemaField("q438", "INTEGER"),
            bigquery.SchemaField("q439", "INTEGER"),
            bigquery.SchemaField("q440", "INTEGER"),
            bigquery.SchemaField("q441", "INTEGER"),
            bigquery.SchemaField("q442", "INTEGER"),
            bigquery.SchemaField("q443", "INTEGER"),
            bigquery.SchemaField("q444", "INTEGER"),
            bigquery.SchemaField("q445", "INTEGER"),
            bigquery.SchemaField("q446", "INTEGER"),
            bigquery.SchemaField("q447", "INTEGER"),
            bigquery.SchemaField("q448", "INTEGER"),
            bigquery.SchemaField("q449", "INTEGER"),
            bigquery.SchemaField("q450", "INTEGER"),
            bigquery.SchemaField("q451", "INTEGER"),
            bigquery.SchemaField("q452", "INTEGER"),
            bigquery.SchemaField("q453", "INTEGER"),
            bigquery.SchemaField("q454", "INTEGER"),
            bigquery.SchemaField("q455", "INTEGER"),
            bigquery.SchemaField("q456", "INTEGER"),
            bigquery.SchemaField("q457", "INTEGER"),
            bigquery.SchemaField("q458", "INTEGER"),
            bigquery.SchemaField("q459", "INTEGER"),
            bigquery.SchemaField("q460", "INTEGER"),
            bigquery.SchemaField("q461", "INTEGER"),
            bigquery.SchemaField("q462", "INTEGER"),
            bigquery.SchemaField("q463", "INTEGER"),
            bigquery.SchemaField("q464", "INTEGER"),
            bigquery.SchemaField("q465", "INTEGER"),
            bigquery.SchemaField("q466", "INTEGER"),
            bigquery.SchemaField("q467", "INTEGER"),
            bigquery.SchemaField("q468", "INTEGER"),
            bigquery.SchemaField("q469", "INTEGER"),
            bigquery.SchemaField("q470", "INTEGER"),
            bigquery.SchemaField("q471", "INTEGER"),
            bigquery.SchemaField("q472", "INTEGER"),
            bigquery.SchemaField("q473", "INTEGER"),
            bigquery.SchemaField("q474", "INTEGER"),
            bigquery.SchemaField("q475", "INTEGER"),
            bigquery.SchemaField("q476", "INTEGER"),
            bigquery.SchemaField("q477", "INTEGER"),
            bigquery.SchemaField("q478", "INTEGER"),
            bigquery.SchemaField("q479", "INTEGER"),
            bigquery.SchemaField("q480", "INTEGER"),
            bigquery.SchemaField("q481", "INTEGER"),
            bigquery.SchemaField("q482", "INTEGER"),
            bigquery.SchemaField("q483", "INTEGER"),
            bigquery.SchemaField("q484", "INTEGER"),
            bigquery.SchemaField("q485", "INTEGER"),
            bigquery.SchemaField("q486", "INTEGER"),
            bigquery.SchemaField("q487", "INTEGER"),
            bigquery.SchemaField("q488", "INTEGER"),
            bigquery.SchemaField("q489", "INTEGER"),
            bigquery.SchemaField("q490", "INTEGER"),
            bigquery.SchemaField("q491", "INTEGER"),
            bigquery.SchemaField("q492", "INTEGER"),
            bigquery.SchemaField("q493", "INTEGER"),
            bigquery.SchemaField("q494", "INTEGER"),
            bigquery.SchemaField("q495", "INTEGER"),
            bigquery.SchemaField("q496", "INTEGER"),
            bigquery.SchemaField("q497", "INTEGER"),
            bigquery.SchemaField("q498", "INTEGER"),
            bigquery.SchemaField("q499", "INTEGER"),
            bigquery.SchemaField("q500", "INTEGER"),
            bigquery.SchemaField("q501", "INTEGER"),
            bigquery.SchemaField("q502", "INTEGER"),
            bigquery.SchemaField("q503", "INTEGER"),
            bigquery.SchemaField("q504", "INTEGER"),
            bigquery.SchemaField("q505", "INTEGER"),
            bigquery.SchemaField("q506", "INTEGER"),
            bigquery.SchemaField("q507", "INTEGER"),
            bigquery.SchemaField("q508", "INTEGER"),
            bigquery.SchemaField("q509", "INTEGER"),
            bigquery.SchemaField("q510", "INTEGER"),
            bigquery.SchemaField("q511", "INTEGER"),
            bigquery.SchemaField("q512", "INTEGER"),
            bigquery.SchemaField("q513", "INTEGER"),
            bigquery.SchemaField("q514", "INTEGER"),
            bigquery.SchemaField("q515", "INTEGER"),
            bigquery.SchemaField("q516", "INTEGER"),
            bigquery.SchemaField("q517", "INTEGER"),
            bigquery.SchemaField("q518", "INTEGER"),
            bigquery.SchemaField("q519", "INTEGER"),
            bigquery.SchemaField("q520", "INTEGER"),
            bigquery.SchemaField("q521", "INTEGER"),
            bigquery.SchemaField("q522", "INTEGER"),
            bigquery.SchemaField("q523", "INTEGER"),
            bigquery.SchemaField("q524", "INTEGER"),
            bigquery.SchemaField("q525", "INTEGER"),
            bigquery.SchemaField("q526", "INTEGER"),
            bigquery.SchemaField("q527", "INTEGER"),
            bigquery.SchemaField("q528", "INTEGER"),
            bigquery.SchemaField("q529", "INTEGER"),
            bigquery.SchemaField("q530", "INTEGER"),
            bigquery.SchemaField("q531", "INTEGER"),
            bigquery.SchemaField("q532", "INTEGER"),
            bigquery.SchemaField("q533", "INTEGER"),
            bigquery.SchemaField("q534", "INTEGER"),
            bigquery.SchemaField("q535", "INTEGER"),
            bigquery.SchemaField("q536", "INTEGER"),
            bigquery.SchemaField("q537", "INTEGER"),
            bigquery.SchemaField("q538", "INTEGER"),
            bigquery.SchemaField("q539", "INTEGER"),
            bigquery.SchemaField("q540", "INTEGER"),
            bigquery.SchemaField("q541", "INTEGER"),
            bigquery.SchemaField("q542", "INTEGER"),
            bigquery.SchemaField("q543", "INTEGER"),
            bigquery.SchemaField("q544", "INTEGER"),
            bigquery.SchemaField("q545", "INTEGER"),
            bigquery.SchemaField("q546", "INTEGER"),
            bigquery.SchemaField("q547", "INTEGER"),
            bigquery.SchemaField("q548", "INTEGER"),
            bigquery.SchemaField("q549", "INTEGER"),
            bigquery.SchemaField("q550", "INTEGER"),
            bigquery.SchemaField("q551", "INTEGER"),
            bigquery.SchemaField("q552", "INTEGER"),
            bigquery.SchemaField("q553", "INTEGER"),
            bigquery.SchemaField("q554", "INTEGER"),
            bigquery.SchemaField("q555", "INTEGER"),
            bigquery.SchemaField("q556", "INTEGER"),
            bigquery.SchemaField("q557", "INTEGER"),
            bigquery.SchemaField("q558", "INTEGER"),
            bigquery.SchemaField("q559", "INTEGER"),
            bigquery.SchemaField("q560", "INTEGER"),
            bigquery.SchemaField("q561", "INTEGER"),
            bigquery.SchemaField("q562", "INTEGER"),
            bigquery.SchemaField("q563", "INTEGER"),
            bigquery.SchemaField("q564", "INTEGER"),
            bigquery.SchemaField("q565", "INTEGER"),
            bigquery.SchemaField("q566", "INTEGER"),
            bigquery.SchemaField("q567", "INTEGER"),
            bigquery.SchemaField("q568", "INTEGER"),
            bigquery.SchemaField("q569", "INTEGER"),
            bigquery.SchemaField("q570", "INTEGER"),
            bigquery.SchemaField("q571", "INTEGER"),
            bigquery.SchemaField("q572", "INTEGER"),
            bigquery.SchemaField("q573", "INTEGER"),
            bigquery.SchemaField("q574", "INTEGER"),
            bigquery.SchemaField("q575", "INTEGER"),
            bigquery.SchemaField("q576", "INTEGER"),
            bigquery.SchemaField("q577", "INTEGER"),
            bigquery.SchemaField("q578", "INTEGER"),
            bigquery.SchemaField("q579", "INTEGER"),
            bigquery.SchemaField("q580", "INTEGER"),
            bigquery.SchemaField("q581", "INTEGER"),
            bigquery.SchemaField("q582", "INTEGER"),
            bigquery.SchemaField("q583", "INTEGER"),
            bigquery.SchemaField("q584", "INTEGER"),
            bigquery.SchemaField("q585", "INTEGER"),
            bigquery.SchemaField("q586", "INTEGER"),
            bigquery.SchemaField("q587", "INTEGER"),
            bigquery.SchemaField("q588", "INTEGER"),
            bigquery.SchemaField("q589", "INTEGER"),
            bigquery.SchemaField("q590", "INTEGER"),
            bigquery.SchemaField("q591", "INTEGER"),
            bigquery.SchemaField("q592", "INTEGER"),
            bigquery.SchemaField("q593", "INTEGER"),
            bigquery.SchemaField("q594", "INTEGER"),
            bigquery.SchemaField("q595", "INTEGER"),
            bigquery.SchemaField("q596", "INTEGER"),
            bigquery.SchemaField("q597", "INTEGER"),
            bigquery.SchemaField("q598", "INTEGER"),
            bigquery.SchemaField("q599", "INTEGER"),
            bigquery.SchemaField("q600", "INTEGER"),
            bigquery.SchemaField("q601", "INTEGER"),
            bigquery.SchemaField("q602", "INTEGER"),
            bigquery.SchemaField("q603", "INTEGER"),
            bigquery.SchemaField("q604", "INTEGER"),
            bigquery.SchemaField("q605", "INTEGER"),
            bigquery.SchemaField("q606", "INTEGER"),
            bigquery.SchemaField("q607", "INTEGER"),
            bigquery.SchemaField("q608", "INTEGER"),
            bigquery.SchemaField("q609", "INTEGER"),
            bigquery.SchemaField("q610", "INTEGER"),
            bigquery.SchemaField("q611", "INTEGER"),
            bigquery.SchemaField("q612", "INTEGER"),
            bigquery.SchemaField("q613", "INTEGER"),
            bigquery.SchemaField("q614", "INTEGER"),
            bigquery.SchemaField("q615", "INTEGER"),
            bigquery.SchemaField("q616", "INTEGER"),
            bigquery.SchemaField("q617", "INTEGER"),
            bigquery.SchemaField("q618", "INTEGER"),
            bigquery.SchemaField("q619", "INTEGER"),
            bigquery.SchemaField("q620", "INTEGER"),
            bigquery.SchemaField("q621", "INTEGER"),
            bigquery.SchemaField("q622", "INTEGER"),
            bigquery.SchemaField("q623", "INTEGER"),
            bigquery.SchemaField("q624", "INTEGER"),
            bigquery.SchemaField("q625", "INTEGER"),
            bigquery.SchemaField("q626", "INTEGER"),
            bigquery.SchemaField("q627", "INTEGER"),
            bigquery.SchemaField("q628", "INTEGER"),
            bigquery.SchemaField("q629", "INTEGER"),
            bigquery.SchemaField("q630", "INTEGER"),
            bigquery.SchemaField("q631", "INTEGER"),
            bigquery.SchemaField("q632", "INTEGER"),
            bigquery.SchemaField("q633", "INTEGER"),
            bigquery.SchemaField("q634", "INTEGER"),
            bigquery.SchemaField("q635", "INTEGER"),
            bigquery.SchemaField("q636", "INTEGER"),
            bigquery.SchemaField("q637", "INTEGER"),
            bigquery.SchemaField("q638", "INTEGER"),
            bigquery.SchemaField("q639", "INTEGER"),
            bigquery.SchemaField("q640", "INTEGER"),
            bigquery.SchemaField("q641", "INTEGER"),
            bigquery.SchemaField("q642", "INTEGER"),
            bigquery.SchemaField("q643", "INTEGER"),
            bigquery.SchemaField("q644", "INTEGER"),
            bigquery.SchemaField("q645", "INTEGER"),
            bigquery.SchemaField("q646", "INTEGER"),
            bigquery.SchemaField("q647", "INTEGER"),
            bigquery.SchemaField("q648", "INTEGER"),
            bigquery.SchemaField("q649", "INTEGER"),
            bigquery.SchemaField("q650", "INTEGER"),
            bigquery.SchemaField("q651", "INTEGER"),
            bigquery.SchemaField("q652", "INTEGER"),
            bigquery.SchemaField("q653", "INTEGER"),
            bigquery.SchemaField("q654", "INTEGER"),
            bigquery.SchemaField("q655", "INTEGER"),
            bigquery.SchemaField("q656", "INTEGER"),
            bigquery.SchemaField("q657", "INTEGER"),
            bigquery.SchemaField("q658", "INTEGER"),
            bigquery.SchemaField("q659", "INTEGER"),
            bigquery.SchemaField("q660", "INTEGER"),
            bigquery.SchemaField("q661", "INTEGER"),
            bigquery.SchemaField("q662", "INTEGER"),
            bigquery.SchemaField("q663", "INTEGER"),
            bigquery.SchemaField("q664", "INTEGER"),
            bigquery.SchemaField("q665", "INTEGER"),
            bigquery.SchemaField("q666", "INTEGER"),
            bigquery.SchemaField("q667", "INTEGER"),
            bigquery.SchemaField("q668", "INTEGER"),
            bigquery.SchemaField("q669", "INTEGER"),
            bigquery.SchemaField("q670", "INTEGER"),
            bigquery.SchemaField("q671", "INTEGER"),
            bigquery.SchemaField("q672", "INTEGER"),
            bigquery.SchemaField("q673", "INTEGER"),
            bigquery.SchemaField("q674", "INTEGER"),
            bigquery.SchemaField("q675", "INTEGER"),
            bigquery.SchemaField("q676", "INTEGER"),
            bigquery.SchemaField("q677", "INTEGER"),
            bigquery.SchemaField("q678", "INTEGER"),
            bigquery.SchemaField("q679", "INTEGER"),
            bigquery.SchemaField("q680", "INTEGER"),
            bigquery.SchemaField("q681", "INTEGER"),
            bigquery.SchemaField("q682", "INTEGER"),
            bigquery.SchemaField("q683", "INTEGER"),
            bigquery.SchemaField("q684", "INTEGER"),
            bigquery.SchemaField("q685", "INTEGER"),
            bigquery.SchemaField("q686", "INTEGER"),
            bigquery.SchemaField("q687", "INTEGER"),
            bigquery.SchemaField("q688", "INTEGER"),
            bigquery.SchemaField("q689", "INTEGER"),
            bigquery.SchemaField("q690", "INTEGER"),
            bigquery.SchemaField("q691", "INTEGER"),
            bigquery.SchemaField("q692", "INTEGER"),
            bigquery.SchemaField("q693", "INTEGER"),
            bigquery.SchemaField("q694", "INTEGER"),
            bigquery.SchemaField("q695", "INTEGER"),
            bigquery.SchemaField("q696", "INTEGER"),
            bigquery.SchemaField("q697", "INTEGER"),
            bigquery.SchemaField("q698", "INTEGER"),
            bigquery.SchemaField("q699", "INTEGER"),
            bigquery.SchemaField("q700", "INTEGER"),
            bigquery.SchemaField("q701", "INTEGER"),
            bigquery.SchemaField("q702", "INTEGER"),
            bigquery.SchemaField("q703", "INTEGER"),
            bigquery.SchemaField("q704", "INTEGER"),
            bigquery.SchemaField("q705", "INTEGER"),
            bigquery.SchemaField("q706", "INTEGER"),
            bigquery.SchemaField("q707", "INTEGER"),
            bigquery.SchemaField("q708", "INTEGER"),
            bigquery.SchemaField("q709", "INTEGER"),
            bigquery.SchemaField("q710", "INTEGER"),
            bigquery.SchemaField("q711", "INTEGER"),
            bigquery.SchemaField("q712", "INTEGER"),
            bigquery.SchemaField("q713", "INTEGER"),
            bigquery.SchemaField("q714", "INTEGER"),
            bigquery.SchemaField("q715", "INTEGER"),
            bigquery.SchemaField("q716", "INTEGER"),
            bigquery.SchemaField("q717", "INTEGER"),
            bigquery.SchemaField("q718", "INTEGER"),
            bigquery.SchemaField("q719", "INTEGER"),
            bigquery.SchemaField("q720", "INTEGER"),
            bigquery.SchemaField("q721", "INTEGER"),
            bigquery.SchemaField("q722", "INTEGER"),
            bigquery.SchemaField("q723", "INTEGER"),
            bigquery.SchemaField("q724", "INTEGER"),
            bigquery.SchemaField("q725", "INTEGER"),
            bigquery.SchemaField("q726", "INTEGER"),
            bigquery.SchemaField("q727", "INTEGER"),
            bigquery.SchemaField("q728", "INTEGER"),
            bigquery.SchemaField("q729", "INTEGER"),
            bigquery.SchemaField("q730", "INTEGER"),
            bigquery.SchemaField("q731", "INTEGER"),
            bigquery.SchemaField("q732", "INTEGER"),
            bigquery.SchemaField("q733", "INTEGER"),
            bigquery.SchemaField("q734", "INTEGER"),
            bigquery.SchemaField("q735", "INTEGER"),
            bigquery.SchemaField("q736", "INTEGER"),
            bigquery.SchemaField("q737", "INTEGER"),
            bigquery.SchemaField("q738", "INTEGER"),
            bigquery.SchemaField("q739", "INTEGER"),
            bigquery.SchemaField("q740", "INTEGER"),
            bigquery.SchemaField("q741", "INTEGER"),
            bigquery.SchemaField("q742", "INTEGER"),
            bigquery.SchemaField("q743", "INTEGER"),
            bigquery.SchemaField("q744", "INTEGER"),
            bigquery.SchemaField("q745", "INTEGER"),
            bigquery.SchemaField("q746", "INTEGER"),
            bigquery.SchemaField("q747", "INTEGER"),
            bigquery.SchemaField("q748", "INTEGER"),
            bigquery.SchemaField("q749", "INTEGER"),
            bigquery.SchemaField("q750", "INTEGER"),
            bigquery.SchemaField("q751", "INTEGER"),
            bigquery.SchemaField("q752", "INTEGER"),
            bigquery.SchemaField("q753", "INTEGER"),
            bigquery.SchemaField("q754", "INTEGER"),
            bigquery.SchemaField("q755", "INTEGER"),
            bigquery.SchemaField("q756", "INTEGER"),
            bigquery.SchemaField("q757", "INTEGER"),
            bigquery.SchemaField("q758", "INTEGER"),
            bigquery.SchemaField("q759", "INTEGER"),
            bigquery.SchemaField("q760", "INTEGER"),
            bigquery.SchemaField("q761", "INTEGER"),
            bigquery.SchemaField("q762", "INTEGER"),
            bigquery.SchemaField("q763", "INTEGER"),
            bigquery.SchemaField("q764", "INTEGER"),
            bigquery.SchemaField("q765", "INTEGER"),
            bigquery.SchemaField("q766", "INTEGER"),
            bigquery.SchemaField("q767", "INTEGER"),
            bigquery.SchemaField("q768", "INTEGER"),
            bigquery.SchemaField("q769", "INTEGER"),
            bigquery.SchemaField("q770", "INTEGER"),
            bigquery.SchemaField("q771", "INTEGER"),
            bigquery.SchemaField("q772", "INTEGER"),
            bigquery.SchemaField("q773", "INTEGER"),
            bigquery.SchemaField("q774", "INTEGER"),
            bigquery.SchemaField("q775", "INTEGER"),
            bigquery.SchemaField("q776", "INTEGER"),
            bigquery.SchemaField("q777", "INTEGER"),
            bigquery.SchemaField("q778", "INTEGER"),
            bigquery.SchemaField("q779", "INTEGER"),
            bigquery.SchemaField("q780", "INTEGER"),
            bigquery.SchemaField("q781", "INTEGER"),
            bigquery.SchemaField("q782", "INTEGER"),
            bigquery.SchemaField("q783", "INTEGER"),
            bigquery.SchemaField("q784", "INTEGER"),
            bigquery.SchemaField("q785", "INTEGER"),
            bigquery.SchemaField("q786", "INTEGER"),
            bigquery.SchemaField("q787", "INTEGER"),
            bigquery.SchemaField("q788", "INTEGER"),
            bigquery.SchemaField("q789", "INTEGER"),
            bigquery.SchemaField("q790", "INTEGER"),
            bigquery.SchemaField("q791", "INTEGER"),
            bigquery.SchemaField("q792", "INTEGER"),
            bigquery.SchemaField("q793", "INTEGER"),
            bigquery.SchemaField("q794", "INTEGER"),
            bigquery.SchemaField("q795", "INTEGER"),
            bigquery.SchemaField("q796", "INTEGER"),
            bigquery.SchemaField("q797", "INTEGER"),
            bigquery.SchemaField("q798", "INTEGER"),
            bigquery.SchemaField("q799", "INTEGER"),
            bigquery.SchemaField("q800", "INTEGER"),
            bigquery.SchemaField("q801", "INTEGER"),
            bigquery.SchemaField("q802", "INTEGER"),
            bigquery.SchemaField("q803", "INTEGER"),
            bigquery.SchemaField("q804", "INTEGER"),
            bigquery.SchemaField("q805", "INTEGER"),
            bigquery.SchemaField("q806", "INTEGER"),
            bigquery.SchemaField("q807", "INTEGER"),
            bigquery.SchemaField("q808", "INTEGER"),
            bigquery.SchemaField("q809", "INTEGER"),
            bigquery.SchemaField("q810", "INTEGER"),
            bigquery.SchemaField("q811", "INTEGER"),
            bigquery.SchemaField("q812", "INTEGER"),
            bigquery.SchemaField("q813", "INTEGER"),
            bigquery.SchemaField("q814", "INTEGER"),
            bigquery.SchemaField("q815", "INTEGER"),
            bigquery.SchemaField("q816", "INTEGER"),
            bigquery.SchemaField("q817", "INTEGER"),
            bigquery.SchemaField("q818", "INTEGER"),
            bigquery.SchemaField("q819", "INTEGER"),
            bigquery.SchemaField("q820", "INTEGER"),
            bigquery.SchemaField("q821", "INTEGER"),
            bigquery.SchemaField("q822", "INTEGER"),
            bigquery.SchemaField("q823", "INTEGER"),
            bigquery.SchemaField("q824", "INTEGER"),
            bigquery.SchemaField("q825", "INTEGER"),
            bigquery.SchemaField("q826", "INTEGER"),
            bigquery.SchemaField("q827", "INTEGER"),
            bigquery.SchemaField("q828", "INTEGER"),
            bigquery.SchemaField("q829", "INTEGER"),
            bigquery.SchemaField("q830", "INTEGER"),
            bigquery.SchemaField("q831", "INTEGER"),
            bigquery.SchemaField("q832", "INTEGER"),
            bigquery.SchemaField("q833", "INTEGER"),
            bigquery.SchemaField("q834", "INTEGER"),
            bigquery.SchemaField("q835", "INTEGER"),
            bigquery.SchemaField("q836", "INTEGER"),
            bigquery.SchemaField("q837", "INTEGER"),
            bigquery.SchemaField("q838", "INTEGER"),
            bigquery.SchemaField("q839", "INTEGER"),
            bigquery.SchemaField("q840", "INTEGER"),
            bigquery.SchemaField("q841", "INTEGER"),
            bigquery.SchemaField("q842", "INTEGER"),
            bigquery.SchemaField("q843", "INTEGER"),
            bigquery.SchemaField("q844", "INTEGER"),
            bigquery.SchemaField("q845", "INTEGER"),
            bigquery.SchemaField("q846", "INTEGER"),
            bigquery.SchemaField("q847", "INTEGER"),
            bigquery.SchemaField("q848", "INTEGER"),
            bigquery.SchemaField("q849", "INTEGER"),
            bigquery.SchemaField("q850", "INTEGER"),
            bigquery.SchemaField("q851", "INTEGER"),
            bigquery.SchemaField("q852", "INTEGER"),
            bigquery.SchemaField("q853", "INTEGER"),
            bigquery.SchemaField("q854", "INTEGER"),
            bigquery.SchemaField("q855", "INTEGER"),
            bigquery.SchemaField("q856", "INTEGER"),
            bigquery.SchemaField("q857", "INTEGER"),
            bigquery.SchemaField("q858", "INTEGER"),
            bigquery.SchemaField("q859", "INTEGER"),
            bigquery.SchemaField("q860", "INTEGER"),
            bigquery.SchemaField("q861", "INTEGER"),
            bigquery.SchemaField("q862", "INTEGER"),
            bigquery.SchemaField("q863", "INTEGER"),
            bigquery.SchemaField("q864", "INTEGER"),
            bigquery.SchemaField("q865", "INTEGER"),
            bigquery.SchemaField("q866", "INTEGER"),
            bigquery.SchemaField("q867", "INTEGER"),
            bigquery.SchemaField("q868", "INTEGER"),
            bigquery.SchemaField("q869", "INTEGER"),
            bigquery.SchemaField("q870", "INTEGER"),
            bigquery.SchemaField("q871", "INTEGER"),
            bigquery.SchemaField("q872", "INTEGER"),
            bigquery.SchemaField("q873", "INTEGER"),
            bigquery.SchemaField("q874", "INTEGER"),
            bigquery.SchemaField("q875", "INTEGER"),
            bigquery.SchemaField("q876", "INTEGER"),
            bigquery.SchemaField("q877", "INTEGER"),
            bigquery.SchemaField("q878", "INTEGER"),
            bigquery.SchemaField("q879", "INTEGER"),
            bigquery.SchemaField("q880", "INTEGER"),
            bigquery.SchemaField("q881", "INTEGER"),
            bigquery.SchemaField("q882", "INTEGER"),
            bigquery.SchemaField("q883", "INTEGER"),
            bigquery.SchemaField("q884", "INTEGER"),
            bigquery.SchemaField("q885", "INTEGER"),
            bigquery.SchemaField("q886", "INTEGER"),
            bigquery.SchemaField("q887", "INTEGER"),
            bigquery.SchemaField("q888", "INTEGER"),
            bigquery.SchemaField("q889", "INTEGER"),
            bigquery.SchemaField("q890", "INTEGER"),
            bigquery.SchemaField("q891", "INTEGER"),
            bigquery.SchemaField("q892", "INTEGER"),
            bigquery.SchemaField("q893", "INTEGER"),
            bigquery.SchemaField("q894", "INTEGER"),
            bigquery.SchemaField("q895", "INTEGER"),
            bigquery.SchemaField("q896", "INTEGER"),
            bigquery.SchemaField("q897", "INTEGER"),
            bigquery.SchemaField("q898", "INTEGER"),
            bigquery.SchemaField("q899", "INTEGER"),
            bigquery.SchemaField("q900", "INTEGER"),
            bigquery.SchemaField("q901", "INTEGER"),
            bigquery.SchemaField("q902", "INTEGER"),
            bigquery.SchemaField("q903", "INTEGER"),
            bigquery.SchemaField("q904", "INTEGER"),
            bigquery.SchemaField("q905", "INTEGER"),
            bigquery.SchemaField("q906", "INTEGER"),
            bigquery.SchemaField("q907", "INTEGER"),
            bigquery.SchemaField("q908", "INTEGER"),
            bigquery.SchemaField("q909", "INTEGER"),
            bigquery.SchemaField("q910", "INTEGER"),
            bigquery.SchemaField("q911", "INTEGER"),
            bigquery.SchemaField("q912", "INTEGER"),
            bigquery.SchemaField("q913", "INTEGER"),
            bigquery.SchemaField("q914", "INTEGER"),
            bigquery.SchemaField("q915", "INTEGER"),
            bigquery.SchemaField("q916", "INTEGER"),
            bigquery.SchemaField("q917", "INTEGER"),
            bigquery.SchemaField("q918", "INTEGER"),
            bigquery.SchemaField("q919", "INTEGER"),
            bigquery.SchemaField("q920", "INTEGER"),
            bigquery.SchemaField("q921", "INTEGER"),
            bigquery.SchemaField("q922", "INTEGER"),
            bigquery.SchemaField("q923", "INTEGER"),
            bigquery.SchemaField("q924", "INTEGER"),
            bigquery.SchemaField("q925", "INTEGER"),
            bigquery.SchemaField("q926", "INTEGER"),
            bigquery.SchemaField("q927", "INTEGER"),
            bigquery.SchemaField("q928", "INTEGER"),
            bigquery.SchemaField("q929", "INTEGER"),
            bigquery.SchemaField("q930", "INTEGER"),
            bigquery.SchemaField("q931", "INTEGER"),
            bigquery.SchemaField("q932", "INTEGER"),
            bigquery.SchemaField("q933", "INTEGER"),
            bigquery.SchemaField("q934", "INTEGER"),
            bigquery.SchemaField("q935", "INTEGER"),
            bigquery.SchemaField("q936", "INTEGER"),
            bigquery.SchemaField("q937", "INTEGER"),
            bigquery.SchemaField("q938", "INTEGER"),
            bigquery.SchemaField("q939", "INTEGER"),
            bigquery.SchemaField("q940", "INTEGER"),
            bigquery.SchemaField("q941", "INTEGER"),
            bigquery.SchemaField("q942", "INTEGER"),
            bigquery.SchemaField("q943", "INTEGER"),
            bigquery.SchemaField("q944", "INTEGER"),
            bigquery.SchemaField("q945", "INTEGER"),
            bigquery.SchemaField("q946", "INTEGER"),
            bigquery.SchemaField("q947", "INTEGER"),
            bigquery.SchemaField("q948", "INTEGER"),
            bigquery.SchemaField("q949", "INTEGER"),
            bigquery.SchemaField("q950", "INTEGER"),
            bigquery.SchemaField("q951", "INTEGER"),
            bigquery.SchemaField("q952", "INTEGER"),
            bigquery.SchemaField("q953", "INTEGER"),
            bigquery.SchemaField("q954", "INTEGER"),
            bigquery.SchemaField("q955", "INTEGER"),
            bigquery.SchemaField("q956", "INTEGER"),
            bigquery.SchemaField("q957", "INTEGER"),
            bigquery.SchemaField("q958", "INTEGER"),
            bigquery.SchemaField("q959", "INTEGER"),
            bigquery.SchemaField("q960", "INTEGER"),
            bigquery.SchemaField("q961", "INTEGER"),
            bigquery.SchemaField("q962", "INTEGER"),
            bigquery.SchemaField("q963", "INTEGER"),
            bigquery.SchemaField("q964", "INTEGER"),
            bigquery.SchemaField("q965", "INTEGER"),
            bigquery.SchemaField("q966", "INTEGER"),
            bigquery.SchemaField("q967", "INTEGER"),
            bigquery.SchemaField("q968", "INTEGER"),
            bigquery.SchemaField("q969", "INTEGER"),
            bigquery.SchemaField("q970", "INTEGER"),
            bigquery.SchemaField("q971", "INTEGER"),
            bigquery.SchemaField("q972", "INTEGER"),
            bigquery.SchemaField("q973", "INTEGER"),
            bigquery.SchemaField("q974", "INTEGER"),
            bigquery.SchemaField("q975", "INTEGER"),
            bigquery.SchemaField("q976", "INTEGER"),
            bigquery.SchemaField("q977", "INTEGER"),
            bigquery.SchemaField("q978", "INTEGER"),
            bigquery.SchemaField("q979", "INTEGER"),
            bigquery.SchemaField("q980", "INTEGER"),
            bigquery.SchemaField("q981", "INTEGER"),
            bigquery.SchemaField("q982", "INTEGER"),
            bigquery.SchemaField("q983", "INTEGER"),
            bigquery.SchemaField("q984", "INTEGER"),
            bigquery.SchemaField("q985", "INTEGER"),
            bigquery.SchemaField("q986", "INTEGER"),
            bigquery.SchemaField("q987", "INTEGER"),
            bigquery.SchemaField("q988", "INTEGER"),
            bigquery.SchemaField("q989", "INTEGER"),
            bigquery.SchemaField("q990", "INTEGER"),
            bigquery.SchemaField("q991", "INTEGER"),
            bigquery.SchemaField("q992", "INTEGER"),
            bigquery.SchemaField("q993", "INTEGER"),
            bigquery.SchemaField("q994", "INTEGER"),
            bigquery.SchemaField("q995", "INTEGER"),
            bigquery.SchemaField("q996", "INTEGER"),
            bigquery.SchemaField("q997", "INTEGER"),
            bigquery.SchemaField("q998", "INTEGER"),
            bigquery.SchemaField("q999", "INTEGER"),
            bigquery.SchemaField("q1000", "INTEGER"),
            bigquery.SchemaField("q1001", "INTEGER"),
            bigquery.SchemaField("q1002", "INTEGER"),
            bigquery.SchemaField("q1003", "INTEGER"),
            bigquery.SchemaField("q1004", "INTEGER"),
            bigquery.SchemaField("q1005", "INTEGER"),
            bigquery.SchemaField("q1006", "INTEGER"),
            bigquery.SchemaField("q1007", "INTEGER"),
            bigquery.SchemaField("q1008", "INTEGER"),
            bigquery.SchemaField("q1009", "INTEGER"),
            bigquery.SchemaField("q1010", "INTEGER"),
            bigquery.SchemaField("q1011", "INTEGER"),
            bigquery.SchemaField("q1012", "INTEGER"),
            bigquery.SchemaField("q1013", "INTEGER"),
            bigquery.SchemaField("q1014", "INTEGER"),
            bigquery.SchemaField("q1015", "INTEGER"),
            bigquery.SchemaField("q1016", "INTEGER"),
            bigquery.SchemaField("q1017", "INTEGER"),
            bigquery.SchemaField("q1018", "INTEGER"),
            bigquery.SchemaField("q1019", "INTEGER"),
            bigquery.SchemaField("q1020", "INTEGER"),
            bigquery.SchemaField("q1021", "INTEGER"),
            bigquery.SchemaField("q1022", "INTEGER"),
            bigquery.SchemaField("q1023", "INTEGER"),
            bigquery.SchemaField("q1024", "INTEGER"),
            bigquery.SchemaField("q1025", "INTEGER"),
            bigquery.SchemaField("q1026", "INTEGER"),
            bigquery.SchemaField("q1027", "INTEGER"),
            bigquery.SchemaField("q1028", "INTEGER"),
            bigquery.SchemaField("q1029", "INTEGER"),
            bigquery.SchemaField("q1030", "INTEGER"),
            bigquery.SchemaField("q1031", "INTEGER"),
            bigquery.SchemaField("q1032", "INTEGER"),
            bigquery.SchemaField("q1033", "INTEGER"),
            bigquery.SchemaField("q1034", "INTEGER"),
            bigquery.SchemaField("q1035", "INTEGER"),
            bigquery.SchemaField("q1036", "INTEGER"),
            bigquery.SchemaField("q1037", "INTEGER"),
            bigquery.SchemaField("q1038", "INTEGER"),
            bigquery.SchemaField("q1039", "INTEGER"),
            bigquery.SchemaField("q1040", "INTEGER"),
            bigquery.SchemaField("q1041", "INTEGER"),
            bigquery.SchemaField("q1042", "INTEGER"),
            bigquery.SchemaField("q1043", "INTEGER"),
            bigquery.SchemaField("q1044", "INTEGER"),
            bigquery.SchemaField("q1045", "INTEGER"),
            bigquery.SchemaField("q1046", "INTEGER"),
            bigquery.SchemaField("q1047", "INTEGER"),
            bigquery.SchemaField("q1048", "INTEGER"),
            bigquery.SchemaField("q1049", "INTEGER"),
            bigquery.SchemaField("q1050", "INTEGER"),
            bigquery.SchemaField("q1051", "INTEGER"),
            bigquery.SchemaField("q1052", "INTEGER"),
            bigquery.SchemaField("q1053", "INTEGER"),
            bigquery.SchemaField("q1054", "INTEGER"),
            bigquery.SchemaField("q1055", "INTEGER"),
            bigquery.SchemaField("q1056", "INTEGER"),
            bigquery.SchemaField("q1057", "INTEGER"),
            bigquery.SchemaField("q1058", "INTEGER"),
            bigquery.SchemaField("q1059", "INTEGER"),
            bigquery.SchemaField("q1060", "INTEGER"),
            bigquery.SchemaField("q1061", "INTEGER"),
            bigquery.SchemaField("q1062", "INTEGER"),
            bigquery.SchemaField("q1063", "INTEGER"),
            bigquery.SchemaField("q1064", "INTEGER"),
            bigquery.SchemaField("q1065", "INTEGER"),
            bigquery.SchemaField("q1066", "INTEGER"),
            bigquery.SchemaField("q1067", "INTEGER"),
            bigquery.SchemaField("q1068", "INTEGER"),
            bigquery.SchemaField("q1069", "INTEGER"),
            bigquery.SchemaField("q1070", "INTEGER"),
            bigquery.SchemaField("q1071", "INTEGER"),
            bigquery.SchemaField("q1072", "INTEGER"),
            bigquery.SchemaField("q1073", "INTEGER"),
            bigquery.SchemaField("q1074", "INTEGER"),
            bigquery.SchemaField("q1075", "INTEGER"),
            bigquery.SchemaField("q1076", "INTEGER"),
            bigquery.SchemaField("q1077", "INTEGER"),
            bigquery.SchemaField("q1078", "INTEGER"),
            bigquery.SchemaField("q1079", "INTEGER"),
            bigquery.SchemaField("q1080", "INTEGER"),
            bigquery.SchemaField("q1081", "INTEGER"),
            bigquery.SchemaField("q1082", "INTEGER"),
            bigquery.SchemaField("q1083", "INTEGER"),
            bigquery.SchemaField("q1084", "INTEGER"),
            bigquery.SchemaField("q1085", "INTEGER"),
            bigquery.SchemaField("q1086", "INTEGER"),
            bigquery.SchemaField("q1087", "INTEGER"),
            bigquery.SchemaField("q1088", "INTEGER"),
            bigquery.SchemaField("q1089", "INTEGER"),
            bigquery.SchemaField("q1090", "INTEGER"),
            bigquery.SchemaField("q1091", "INTEGER"),
            bigquery.SchemaField("q1092", "INTEGER"),
            bigquery.SchemaField("q1093", "INTEGER"),
            bigquery.SchemaField("q1094", "INTEGER"),
            bigquery.SchemaField("q1095", "INTEGER"),
            bigquery.SchemaField("q1096", "INTEGER"),
            bigquery.SchemaField("q1097", "INTEGER"),
            bigquery.SchemaField("q1098", "INTEGER"),
            bigquery.SchemaField("q1099", "INTEGER"),
            bigquery.SchemaField("q1100", "INTEGER"),
            bigquery.SchemaField("q1101", "INTEGER"),
            bigquery.SchemaField("q1102", "INTEGER"),
            bigquery.SchemaField("q1103", "INTEGER"),
            bigquery.SchemaField("q1104", "INTEGER"),
            bigquery.SchemaField("q1105", "INTEGER"),
            bigquery.SchemaField("q1106", "INTEGER"),
            #does not exist
            #bigquery.SchemaField("q1107", "INTEGER"),
            bigquery.SchemaField("q1108", "INTEGER"),
            bigquery.SchemaField("q1109", "INTEGER"),
            bigquery.SchemaField("q1110", "INTEGER"),
            bigquery.SchemaField("q1111", "INTEGER"),
            bigquery.SchemaField("q1112", "INTEGER"),
            bigquery.SchemaField("q1113", "INTEGER"),
            bigquery.SchemaField("q1114", "INTEGER"),
            bigquery.SchemaField("q1115", "INTEGER"),
            bigquery.SchemaField("q1116", "INTEGER"),
            bigquery.SchemaField("q1117", "INTEGER"),
            bigquery.SchemaField("q1118", "INTEGER"),
            bigquery.SchemaField("q1119", "INTEGER"),
            bigquery.SchemaField("q1120", "INTEGER"),
            bigquery.SchemaField("q1121", "INTEGER"),
            bigquery.SchemaField("q1122", "INTEGER"),
            bigquery.SchemaField("q1123", "INTEGER"),
            bigquery.SchemaField("q1124", "INTEGER"),
            bigquery.SchemaField("q1125", "INTEGER"),
            bigquery.SchemaField("q1126", "INTEGER"),
            bigquery.SchemaField("q1127", "INTEGER"),
            bigquery.SchemaField("q1128", "INTEGER"),
            bigquery.SchemaField("q1129", "INTEGER"),
            bigquery.SchemaField("q1130", "INTEGER"),
            bigquery.SchemaField("q1131", "INTEGER"),
            bigquery.SchemaField("q1132", "INTEGER"),
            bigquery.SchemaField("q1133", "INTEGER"),
            bigquery.SchemaField("q1134", "INTEGER"),
            bigquery.SchemaField("q1135", "INTEGER"),
            bigquery.SchemaField("q1136", "INTEGER"),
            bigquery.SchemaField("q1137", "INTEGER"),
            bigquery.SchemaField("q1138", "INTEGER"),
            bigquery.SchemaField("q1139", "INTEGER"),
            bigquery.SchemaField("q1140", "INTEGER"),
            bigquery.SchemaField("q1141", "INTEGER"),
            bigquery.SchemaField("q1142", "INTEGER"),
            bigquery.SchemaField("q1143", "INTEGER"),
            bigquery.SchemaField("q1144", "INTEGER"),
            bigquery.SchemaField("q1145", "INTEGER"),
            bigquery.SchemaField("q1146", "INTEGER"),
            bigquery.SchemaField("q1147", "INTEGER"),
            bigquery.SchemaField("q1148", "INTEGER"),
            bigquery.SchemaField("q1149", "INTEGER"),
            bigquery.SchemaField("q1150", "INTEGER"),
            bigquery.SchemaField("q1151", "INTEGER"),
            bigquery.SchemaField("q1152", "INTEGER"),
            bigquery.SchemaField("q1153", "INTEGER"),
            bigquery.SchemaField("q1154", "INTEGER"),
            bigquery.SchemaField("q1155", "INTEGER"),
            bigquery.SchemaField("q1156", "INTEGER"),
            bigquery.SchemaField("q1157", "INTEGER"),
            bigquery.SchemaField("q1158", "INTEGER"),
            bigquery.SchemaField("q1159", "INTEGER"),
            bigquery.SchemaField("q1160", "INTEGER"),
            bigquery.SchemaField("q1161", "INTEGER"),
            bigquery.SchemaField("q1162", "INTEGER"),
            bigquery.SchemaField("q1163", "INTEGER")
        ],
        # Set job configuration write_disposition to be able to overwrite existing table
    write_disposition='WRITE_TRUNCATE'
    )

    # Define job
job = client.load_table_from_dataframe(
    muse_model_df,
    table_id,
    job_config=job_config,
    location='US'
)

    # Wait for the load job to complete.
job.result()
    
    # Make an API request
table = client.get_table(table_id)
print(
    'Loaded {} rows with {} columns to {}'.format(table.num_rows, len(table.schema), table_id)
    )

KeyError: 'q1'